# Zip by zip forecast

## Set up

In [1]:
# DataFrame Management
import pandas as pd

# Maths and statistics
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.graphics.tsaplots import plot_acf

# Plotting
from matplotlib import pyplot as plt

# Strings
import re

# Progress monitoring
from tqdm import tqdm

# Path management
import pathlib
import os
import platform

# Date Management
import datetime
from datetime import timedelta

In [2]:
# Set up path
path = pathlib.Path().resolve()

# If linux, different
if platform.system() != 'Linux':
    data_path = path.parent / "Dropbox" / "DOT_Tobin_Collaboration" / "data"
else:
    project_path = path.parent.parent / "rn_home"
    data_path = project_path / "data" / "rlpolk_data"

# Prepare Data

### Import Data

In [3]:
# Import raw data to be used
vehicle_sales = pd.read_csv(data_path / "new_vehicle_sales_month_year_zip.csv")
vehicle_sales = vehicle_sales.drop("Unnamed: 0", axis = 1)
vehicle_sales_filled = pd.read_csv(data_path / "vehicle_sales_filled.csv", index_col = [0])

In [7]:
total_by_zip_yr_mth = vehicle_sales_filled.groupby(["ZIP_CODE", "year", "month"]).sum()[["VEH_COUNT"]].reset_index()
total_by_yr_mth = vehicle_sales_filled.groupby(["year", "month"]).sum()[["VEH_COUNT"]].reset_index()

In [8]:
# Add the total vehicles sold
vehicle_sales_filled = vehicle_sales_filled.merge(total_by_zip_yr_mth, 
                                                  left_on = ["ZIP_CODE", "year", "month"], 
                                                  right_on = ["ZIP_CODE", "year", "month"],
                                                  how = 'left').rename(columns = {"VEH_COUNT_y" : "TOTAL_VEH_COUNT",
                                                                                 "VEH_COUNT_x" : "VEH_COUNT"})

In [9]:
# Add the total vehicles sold
vehicle_sales_filled = vehicle_sales_filled.merge(total_by_yr_mth, 
                                                  left_on = ["year", "month"], 
                                                  right_on = ["year", "month"],
                                                  how = 'left').rename(columns = {"VEH_COUNT_y" : "TOTAL_VEH_COUNT_YEAR_MONTH",
                                                                                 "VEH_COUNT_x" : "VEH_COUNT"})

In [11]:
# Calculate the share
def f(veh_cnt, total_veh_cnt):
    try: 
        if total_veh_cnt == 0:
            return 0
        else:
            return round(veh_cnt / total_veh_cnt,4)
    except:
        return np.NaN

vehicle_sales_filled["share_in_zip_yr_mth"] = vehicle_sales_filled.apply(lambda x: f(x.VEH_COUNT, x.TOTAL_VEH_COUNT), axis = 1)
vehicle_sales_filled["share_in_yr_mth"] = vehicle_sales_filled.apply(lambda x: f(x.VEH_COUNT, x.TOTAL_VEH_COUNT_YEAR_MONTH), axis = 1)

In [153]:
vehicle_sales_filled.loc[:, "day"] = 1
vehicle_sales_filled.loc[:,"date"] = pd.to_datetime(ev_sales_filled[["year", "month", "day"]])

In [13]:
# Create EV sales data with zeroes filled (we create a date column to ease plotting)
ev_sales_filled = vehicle_sales_filled.loc[vehicle_sales_filled[vehicle_sales_filled["FuelTypePrimary"] == "Electric"].index, :]
ev_sales_filled.loc[:, "day"] = 1
ev_sales_filled.loc[:,"date"] = pd.to_datetime(ev_sales_filled[["year", "month", "day"]])

### Create out-of-sample data to be used for prediction later

In [15]:
# Set up prediction data
predict_years = [2018, 2019, 2020, 2021, 2022,2023, 2024, 2025, 2026,2027,2028,2029,2030]
predict_months = [1,2,3,4,5,6,7,8,9,10,11,12]
predict_data = pd.DataFrame([])

i = 0
for year in predict_years:
    for month in predict_months:
        predict_data.loc[i, "year"] = int(year)
        predict_data.loc[i, "month"] = f"{int(month)}".zfill(2)
        predict_data.loc[i, "time"] = int(i)
        i+=1
    
predict_month_dummies = pd.get_dummies(predict_data["month"], prefix = 'month')
predict_data = pd.concat([predict_data, predict_month_dummies], axis = 1)
predict_data["day"] = 1

dates = pd.to_datetime(predict_data[["year", "month", "day"]])

predict_data.loc[dates > datetime.datetime(2020,7,1), "post_jun_2020"] = True
predict_data.loc[~(dates > datetime.datetime(2020,7,1)), "post_jun_2020"] = False

# Prepare data
predict_data["time2"]=predict_data["time"]**2

/tmp/ipykernel_2326829/2883918726.py:10: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '01' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  predict_data.loc[i, "month"] = f"{int(month)}".zfill(2)
/tmp/ipykernel_2326829/2883918726.py:20: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'True' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  predict_data.loc[dates > datetime.datetime(2020,7,1), "post_jun_2020"] = True


# Model zip by zip

### Modelling

In [69]:
quad_model_dict = {}
quad_share_model_dict = {}

quad_predictions_dict = {}
quad_share_predictions_dict = {}

# lin_errors_dict = {}
# quad_errors_dict = {}

# lin_forecast_dict = {}
quad_forecast_dict = {}
quad_share_forecast_dict = {}
logit_share_forecast_dict = {}

observed_dict = {}

failed_zips = []

In [71]:
def model_by_zip(input_df, zip_codes):
    # Define the formulas
    form_lin = 'VEH_COUNT~time+month_01+month_02+month_03+month_04+month_05+month_06+month_07+month_08+month_09+month_10+month_11+month_12'
    form_quad = 'VEH_COUNT~time2+month_01+month_02+month_03+month_04+month_05+month_06+month_07+month_08+month_09+month_10+month_11+month_12'
    form_lin_share = 'share_in_zip_yr_mth~time+month_01+month_02+month_03+month_04+month_05+month_06+month_07+month_08+month_09+month_10+month_11+month_12'
    form_quad_share = 'share_in_zip_yr_mth~time2+month_01+month_02+month_03+month_04+month_05+month_06+month_07+month_08+month_09+month_10+month_11+month_12'
    
    for zip_code in tqdm(zip_codes):
        try:
            # Create a dataframe containing a running time variable
            df = input_df[input_df["ZIP_CODE"]==zip_code][["VEH_COUNT", "TOTAL_VEH_COUNT", "share_in_zip_yr_mth", "year", "month", "day", "date"]]
            df = df.sort_values("date", ascending=True).reset_index().reset_index().rename(columns = {"level_0":"time"}).drop("index", axis=1)
            
            # Create dummies
            df["month"] = df["date"].astype(str).str[5:7]
            month_dummies = pd.get_dummies(df["month"], prefix = 'month')
            df = pd.concat([df, month_dummies], axis =1)
            
            # Create the quadratic time data
            df["time2"] = df["time"]**2
            
            # Estimate the quadratic model
            zip_model_quad = smf.ols(formula=form_quad, data = df)
            zip_model_quad_results = zip_model_quad.fit()

            # Estimate quadratic with shares
            zip_model_quad_share = smf.ols(formula=form_quad_share, data = df)
            zip_model_quad_share_results = zip_model_quad_share.fit()

            # Estimate logit
            form_logit = 'share_in_zip_yr_mth~time'
            logit_model = smf.logit(form_logit, data = df)
            logit_model_results = logit_model.fit(disp = 0)

            # Add the linear and quadratic models to the respective dictionaries
            quad_model_dict[zip_code] = zip_model_quad_results
            quad_share_model_dict[zip_code] = zip_model_quad_share_results
            
            # Get quadratic forecast to 2030
            forecast_quadratic = zip_model_quad_results.predict(predict_data)
            quad_forecast_dict[zip_code] = forecast_quadratic

            # Get quadratic share forecast to 2030
            forecast_quadratic_share = zip_model_quad_share_results.predict(predict_data)
            quad_share_forecast_dict[zip_code] = forecast_quadratic_share

            # Get logistic share forecast 
            forecast_logit_share = logit_model_results.predict(predict_data)
            logit_share_forecast_dict[zip_code] = forecast_logit_share
            
            # Get real
            observed_dict[zip_code] = df[["year", "month", "VEH_COUNT"]]
        
        except Exception as e:
            failed_zips.append(zip_code)
        

# Save

In [72]:
# Transform the predictions into long format and save as a csv
zip_by_zip_predictions_lin = pd.DataFrame(lin_predictions_dict)
value_vars = pd.DataFrame(lin_predictions_dict).T.index
zip_by_zip_predictions_lin = pd.melt(pd.DataFrame(lin_predictions_dict).T.reset_index(), id_vars = 'index').rename(columns = {"index":"zip",
                                                                                  "variable":"time"}).sort_values(["zip","time"]).reset_index(drop=True)

In [73]:
# Create quad predictions
zip_by_zip_predictions_quad = pd.DataFrame(quad_predictions_dict)
value_vars = pd.DataFrame(quad_predictions_dict).T.index
zip_by_zip_predictions_quad = pd.melt(pd.DataFrame(quad_predictions_dict).T.reset_index(), id_vars = 'index').rename(columns = {"index":"zip",
                                                                                  "variable":"time"}).sort_values(["zip","time"]).reset_index(drop=True)

In [74]:
# Create quadratic forecast
quad_forecast = pd.melt(pd.DataFrame(quad_forecast_dict).reset_index().rename(columns = {"index":"time"}), id_vars = 'time')
quad_forecast = quad_forecast.rename(columns = {"variable":"zip"}).sort_values(["zip", "time"]).reset_index(drop=True)

quad_forecast.groupby("time").sum()["value"].sum()

283226.6646530895

In [87]:
logit_forecast = pd.melt(pd.DataFrame(logit_share_forecast_dict).reset_index().rename(columns = {"index":"time"}), id_vars = 'time')

In [121]:
quad_share_forecast = pd.melt(pd.DataFrame(quad_share_forecast_dict).reset_index().rename(columns = {"index":"time"}), id_vars = 'time')

# Save

In [126]:
logit_forecast.to_csv(data_path / "zip-level_logit_forecast.csv")
quad_share_forecast.to_csv(data_path / "zip-level_quad_forecast.csv")

# Quickly estimate sales per zip per year-month

In [151]:
agg_forecast_lin = {}

In [169]:
total_by_zip_mth = total_by_zip_yr_mth.groupby(["ZIP_CODE", "year"]).sum().reset_index()[["ZIP_CODE", "year", "VEH_COUNT"]]

In [179]:
total_by_zip_mth_wide = total_by_zip_mth.pivot(index = 'ZIP_CODE', columns = 'year', values = 'VEH_COUNT').reset_index()

In [192]:
total_by_zip_mth_wide.to_csv(data_path / "agg_sales_zip_mth.csv")

In [189]:
total_by_zip_mth_wide.iloc[:, [5+1]]

IndexError: positional indexers are out-of-bounds

In [185]:
total_by_zip_mth_wide.iloc[:, [5]]

year,2022
0,925
1,708
2,5
3,1992
4,458
...,...
347,1093
348,429
349,527
350,0


In [ ]:
df.pivot(index='patient', columns='obs', values='score')

**We visualize the in-sample predictions for the top 10 zip codes by sales of all vehicles**

In [110]:
top_10_zips = total_sales_by_zip.groupby("ZIP_CODE").sum().sort_values("TOTAL_VEH_COUNT",ascending=False).head(10).reset_index()["ZIP_CODE"].to_list()